In [1]:
from IPython.display import display
import ipywidgets as widgets
from IPython.display import clear_output
import numpy as np
from ipywidgets import Video, Image
from glob import glob
import os

import time
import cv2
import time
import matplotlib.pyplot as plt
import subprocess, shlex
import PIL as pil
import random
import functools
import pandas as pd
import math

In [9]:
def add_radio_button():
    toggle = widgets.RadioButtons(
        options = ['YES', 'NO'],
        value = None,
        layout = {'width': 'max-content'}, 
        description = 'Similar? : \n',
        disabled = False
    )
    return toggle


def next_item(text_path, root_path, index): 
    clear_output()
    index+=1
    display_image(text_path, root_path, index, save_name)

def click_previous(arg, argument):
    clear_output()
    text_path, root_path, index, save_name = arg
    with open(f'{save_name}.txt', 'r+') as f:
        # read an store all lines into list
        lines = f.readlines()
        f.seek(0)
        f.truncate()
        f.writelines(lines[:-1])
    index-=1
    display_image(text_path, root_path, index, save_name)

def click_no(arg, argument):
    try:
        text_path, root_path, index, save_name = arg
        img_1,img_2,img_3 = get_img_paths(text_path, index)  
        with open(f'{save_name}.txt', 'a') as file:
            file.write(f"{img_1}|{img_2}|{img_3}|1\n")
        print("recorded")
        next_item(text_path, root_path, index)
    except Exception as e:
        print(e)
        print("CONGRATS...TASK FINITO !!")
    
def click_yes(arg, argument):
    try: 
        text_path, root_path, index, save_name = arg
        img_1,img_2,img_3 = get_img_paths(text_path, index)  
        with open(f'{save_name}.txt', 'a') as file:
            file.write(f"{img_1}|{img_2}|{img_3}|0\n")
        print("recorded")
        next_item(text_path, root_path, index)
    except Exception as e:
        print(e)
        print("CONGRATS...TASK FINITO !!")
    

def get_img_paths(text_path, index):
    with open(text_path, 'r') as f:
        paths = f.readlines()[index]
        img_1 = paths.split('|')[0]
        img_2 = paths.split('|')[1]
        img_3 = paths.split('|')[2]            
    return img_1, img_2, img_3

def display_image(text_path, root_path, index, save_name):
    with open('log.txt', 'w') as f:
        f.write(str(index))

    #images
    img_1,img_2,img_3 = get_img_paths(text_path, index)  
#     print(img_1, 'and', img_3)

    image_1           = cv2.resize(cv2.imread(os.path.join(root_path, img_1)), (300, 300))
    image_3           = cv2.resize(cv2.imread(os.path.join(root_path, img_3)), (300, 300))
    concated          = cv2.hconcat([image_1, image_3])
    cv2.imwrite('temp_file.png', concated)
    images            = Image.from_file('temp_file.png',  width = 600, height=600)

    #buttons
    arg    = text_path, root_path, index, save_name
    yes = widgets.Button(description = 'Yes', disabled=False)
    yes.on_click(functools.partial(click_yes, arg))
    
    no = widgets.Button(description = 'No', disabled=False)
    no.on_click(functools.partial(click_no, arg))
    
    previous = widgets.Button(description = 'Previous', disabled=False)
    previous.on_click(functools.partial(click_previous, arg))
    
    #display all
    content = widgets.HBox([images])
    radio = widgets.HBox([yes, no])
    buttons = widgets.HBox([previous])
    display(widgets.VBox([content, radio, buttons , widgets.Output()]))


In [10]:
ind_number = 0
current_text_path = f'roshan_{ind_number}.txt'

root_path          = ''
classes            = 'long_sleeve_outwear_0_new'
save_name          = f'{classes}_{ind_number}'
original_text_path = f'{classes}.txt'

df = pd.read_csv(original_text_path, sep = '|', header = None)
div_length    = math.ceil(df.shape[0]/6)

read_orig     = open(original_text_path)
content       = read_orig.readlines()[div_length * ind_number : (div_length * ind_number) + div_length]

with open(current_text_path, 'w') as j:
    j.writelines(content)
    j.close()

print(div_length * ind_number)
print((div_length * ind_number) + div_length)

0
3686


In [11]:
try:
    with open('log.txt', 'r') as f:
        index = int(f.readline())      
except Exception as e:
    index = 0

display_image(current_text_path,root_path, index, save_name)